In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv("../Datasets/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [4]:
df.loc[df.duplicated()]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [6]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [1]:
#!conda install -c conda-forge imbalanced-learn --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: D:\conda\envs\Kevin X DS

  added / updated specs:
    - imbalanced-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    imbalanced-learn-0.8.0     |     pyhd8ed1ab_0         109 KB  conda-forge
    openssl-1.1.1k             |       h8ffe710_0         5.7 MB  conda-forge
    scikit-learn-0.24.1        |   py38hf11a4ad_0         4.8 MB
    ------------------------------------------------------------
                                           Total:        10.7 MB

The following NEW packages will be INSTALLED:

  imbalanced-learn   conda-forge/noarch::imbalanced-learn-0.8.0-pyhd8ed1ab_0

The following packages will be UPDATED:

  certifi            pkgs/main::certifi-2020.12.5-py38haa9~ --> conda-forge::certifi-2020.12.5-py38haa244fe_1
  openssl              pkgs/main::openssl-1.1.1j-h2bbff1b_0 --> con

In [28]:
df.columns[(df.dtypes ==  np.int64) | (df.dtypes == np.float64)]

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [156]:
from sklearn.model_selection import train_test_split,GridSearchCV,cross_validate
#from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer,precision_recall_fscore_support,accuracy_score,roc_auc_score
from imblearn.pipeline import Pipeline # use the imblearn if incase you want to use sampler
from imblearn.over_sampling import BorderlineSMOTE,SMOTE

In [61]:
X = df.drop('Outcome',axis=1)
y = df['Outcome']

In [62]:
len(X)

768

In [63]:
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [129]:
#stratify is a must when dealing with imbalance dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42,stratify=y)

In [134]:
smote = SMOTE(random_state=42)
scaler = StandardScaler()
logreg = LogisticRegression(max_iter=50000)

log_reg = Pipeline(steps=[
    ('scale',scaler),
    ('smote',smote),
    ('logreg',logreg)
])

lr_param = {'logreg__C':np.array([0.01,0.5,1,3,5,10]),
            'logreg__solver':['saga','lbfgs']}
lr_search = GridSearchCV(log_reg,lr_param)
lr_search.fit(X_train,y_train)
yhat_lrl2 = pipe.predict(X_test)

In [137]:
print(lr_search.best_params_)
print(lr_search.best_score_)
print(classification_report(y_test,yhat_lrl2))

{'logreg__C': 1.0, 'logreg__solver': 'saga'}
0.9962500000000001
              precision    recall  f1-score   support

           0       1.00      0.90      0.95        20
           1       0.99      1.00      0.99       180

    accuracy                           0.99       200
   macro avg       0.99      0.95      0.97       200
weighted avg       0.99      0.99      0.99       200



In [139]:
log_reg_l1 = Pipeline(steps=[
    ('scale',scaler),
    ('smote',smote),
    ('l1',LogisticRegression(penalty='l1',max_iter=50000))
])

l1_param = {'l1__C':np.array([0.01,0.5,1,3,5,10]),
           'l1__solver':['saga','lbfgs']}
l1_search = GridSearchCV(log_reg_l1,param_grid=l1_param)
l1_search.fit(X_train,y_train)

yhat_lrl1 = l1_search.predict(X_test)

D:\conda\envs\Kevin X DS\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\conda\envs\Kevin X DS\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\conda\envs\Kevin X DS\lib\site-packages\imblearn\pipeline.py", line 266, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "D:\conda\envs\Kevin X DS\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "D:\conda\envs\Kevin X DS\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, go

In [141]:
print(l1_search.best_params_)
print(l1_search.best_score_)

{'l1__C': 0.5, 'l1__solver': 'saga'}
0.9962500000000001


In [145]:
logreg_l3 = Pipeline(steps=[
    ('scale',scaler),
    ('smote',smote),
    ('l3',LogisticRegression(penalty='elasticnet',max_iter=50000,solver='saga'))
])
logreg_l3_param = {'l3__C':np.array([0.01,0.5,1,3,5,10]),
                  'l3__l1_ratio':np.linspace(0,1,10)}
l3_search = GridSearchCV(logreg_l3,param_grid=logreg_l3_param)
l3_search.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('smote', SMOTE(random_state=42)),
                                       ('l3',
                                        LogisticRegression(max_iter=50000,
                                                           penalty='elasticnet',
                                                           solver='saga'))]),
             param_grid={'l3__C': array([ 0.01,  0.5 ,  1.  ,  3.  ,  5.  , 10.  ]),
                         'l3__l1_ratio': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])})

In [147]:
print(l3_search.best_params_)
print(l3_search.best_score_)

{'l3__C': 0.5, 'l3__l1_ratio': 1.0}
0.9962500000000001


In [148]:
yhat_l3 = l3_search.predict(X_test)
print(classification_report(y_test,yhat_l3))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95        20
           1       0.99      1.00      0.99       180

    accuracy                           0.99       200
   macro avg       0.99      0.95      0.97       200
weighted avg       0.99      0.99      0.99       200



In [154]:
y_pred_all = []
y_prob_all = []
model_labels = ['l1','l2','l3']
models = [l1_search,lr_search,l3_search]

for i,j in zip(model_labels,models):
    y_pred_all.append(pd.Series(j.predict(X_test),name=i))
    y_prob_all.append(pd.Series(j.predict_proba(X_test).max(axis=1),name=i))
    
y_pred_all = pd.concat(y_pred_all,axis=1)
y_prob_all = pd.concat(y_prob_all,axis=1)

In [153]:
y_pred_all

[0      1
 1      0
 2      1
 3      1
 4      0
       ..
 195    1
 196    1
 197    1
 198    1
 199    1
 Name: l1, Length: 200, dtype: int32,
 0      1
 1      0
 2      1
 3      1
 4      0
       ..
 195    1
 196    1
 197    1
 198    1
 199    1
 Name: l2, Length: 200, dtype: int32,
 0      1
 1      0
 2      1
 3      1
 4      0
       ..
 195    1
 196    1
 197    1
 198    1
 199    1
 Name: l3, Length: 200, dtype: int32]

In [155]:
y_pred_all

,l1,l2,l3
0,1,1,1
1,0,0,0
2,1,1,1
3,1,1,1
4,0,0,0
...,...,...,...
195,1,1,1
196,1,1,1
197,1,1,1
198,1,1,1


In [163]:
metrics = []

for i in model_labels:
    precision,recall,fscore,_ = precision_recall_fscore_support(y_test,y_pred_all[i],average='weighted')
    
    accuracy = accuracy_score(y_test,y_pred_all[i])
    
    auc = roc_auc_score(y_test,y_pred_all[i],average='weighted')
    
    metrics.append(pd.Series({'precision':precision,'recall':recall,
                             'f1_score':fscore,'accuracy':accuracy,
                             'auc':auc},name=i))
    


In [164]:
metrics = pd.concat(metrics,axis=1)

In [165]:
metrics

,l1,l2,l3
precision,0.990110,0.990110,0.990110
recall,0.990000,0.990000,0.990000
f1_score,0.989764,0.989764,0.989764
accuracy,0.990000,0.990000,0.990000
auc,0.950000,0.950000,0.950000
